## Feature Selection with Mutual Information

The aim of this notebook:
- Look at column features and how they influence a cell's self-exciting property (mutual info)

In [1]:
import os
import logging as log
from time import strftime
from copy import deepcopy
from torch import nn, optim
import torch.nn.functional as F
from utils.data_processing import *
from logger.logger import setup_logging
from utils.configs import BaseConf
from utils.utils import write_json, Timer, get_data_sub_paths, pshape, get_data_resolutions
from models.kangkang_fnn_models import KangFeedForwardNetwork, SimpleKangFNN
from dataloaders.flat_loader import FlatDataLoaders, MockLoader, MockLoaders
from datasets.flat_dataset import FlatDataGroup

from dataloaders.cell_loader import CellDataLoaders
from datasets.cell_dataset import CellDataGroup

from utils.metrics import PRCurvePlotter, ROCCurvePlotter, LossPlotter, PerTimeStepPlotter
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
from models.model_result import ModelResult, ModelMetrics, save_metrics
from utils.mock_data import mock_fnn_data_classification
from utils.plots import im
from trainers.generic_trainer import train_model
from models.kangkang_fnn_models import train_epoch_for_fnn
from utils.configs import BaseConf
from utils.metrics import best_threshold, get_y_pred
from dataloaders.grid_loader import GridDataLoaders
from datasets.grid_dataset import GridDataGroup

from utils.mock_data import mock_fnn_data_classification
import matplotlib.pyplot as plt
from utils.plots import im
from utils.metrics import best_threshold, get_y_pred, get_y_pred_by_thresholds, best_thresholds
from models.model_result import ModelResult, ModelMetrics, save_metrics, compare_all_models,\
                                get_models_metrics, get_models_results
from pprint import pprint
from time import time
from utils.setup import setup

In [20]:
conf = BaseConf()

conf.model_name = f"Mutual Info"

conf.data_path = f"./data/processed/{data_sub_path}/"

if not os.path.exists(conf.data_path):
    raise Exception(f"Directory ({conf.data_path}) needs to exist.")

conf.model_path = f"{conf.data_path}models/{conf.model_name}/"
os.makedirs(conf.data_path, exist_ok=True)
os.makedirs(conf.model_path, exist_ok=True)

setup_logging(save_dir=conf.model_path,
              log_config='./logger/standard_logger_config.json',
              default_level=log.INFO)

with np.load(conf.data_path + "generated_data.npz") as zip_file:  # context helper ensures zip_file is closed
    sparse_crimes = zip_file["crime_types_grids"]
    crime_feature_indices = zip_file["crime_feature_indices"]


shaper = Shaper(data=sparse_crimes,
                conf=conf)

#     return conf, shaper, sparse_crimes, crime_feature_indices


In [2]:
data_sub_paths = get_data_sub_paths()
pprint(np.sort(data_sub_paths))

data_sub_path = 'T24H-X850M-Y880M_2013-01-01_2017-01-01'

array(['T12H-X850M-Y880M_2013-01-01_2017-01-01',
       'T1H-X1700M-Y1760M_2013-01-01_2017-01-01',
       'T24H-X1275M-Y1320M_2012-01-01_2019-01-01',
       'T24H-X1700M-Y1760M_2012-01-01_2019-01-01',
       'T24H-X255M-Y220M_2013-01-01_2017-01-01',
       'T24H-X425M-Y440M_2012-01-01_2019-01-01',
       'T24H-X425M-Y440M_2013-01-01_2017-01-01',
       'T24H-X850M-Y880M_2012-01-01_2019-01-01',
       'T24H-X850M-Y880M_2013-01-01_2017-01-01',
       'T24H-X85M-Y110M_2013-01-01_2017-01-01',
       'T3H-X850M-Y880M_2013-01-01_2017-01-01',
       'T6H-X850M-Y880M_2013-01-01_2017-01-01'], dtype='<U40')


In [3]:
conf, shaper, sparse_crimes, crime_feature_indices = setup(data_sub_path=data_sub_path)

In [4]:
from utils.preprocessing import Shaper

In [5]:
for i,k in enumerate(crime_feature_indices):
    print(f"'{k}':{i},")

'TOTAL':0,
'THEFT':1,
'BATTERY':2,
'CRIMINAL DAMAGE':3,
'NARCOTICS':4,
'ASSAULT':5,
'BURGLARY':6,
'MOTOR VEHICLE THEFT':7,
'ROBBERY':8,
'Arrest':9,


In [6]:
i = 0
crimes = sparse_crimes[:,i:i+1]
print(conf.shaper_threshold) # sum over all time should be above this threshold
print(conf.shaper_top_k) # if larger than 0 we filter out only the top k most active cells of the data grid
new_shaper = Shaper(crimes, conf)

0
-1


In [11]:
dense_crimes = shaper.squeeze(sparse_crimes)
from utils.utils import describe_array

In [19]:
print(describe_array(dense_crimes[:,4]))

____________________________________________________________
{'max': 43.0,
 'mean': 0.084798899185383,
 'min': 0.0,
 'shape': (1461, 772),
 'std': 0.37116642461852944}
____________________________________________________________


In [56]:
dense_crimes[dense_crimes > 0] = 1

In [57]:
c_now = dense_crimes[:-1,1:-1]
c_prev = dense_crimes[1:,1:-1]

In [58]:
c_now_flt = np.reshape(c_now.swapaxes(1,2),(-1,8))
c_prev_flt = np.reshape(c_prev.swapaxes(1,2),(-1,8))

In [59]:
df_now = pd.DataFrame(c_now_flt,columns=crime_feature_indices[1:-1])
df_prev = pd.DataFrame(c_prev_flt,columns=crime_feature_indices[1:-1])

In [60]:
df_now.corr()

,THEFT,BATTERY,CRIMINAL DAMAGE,NARCOTICS,ASSAULT,BURGLARY,MOTOR VEHICLE THEFT,ROBBERY
THEFT,1.000000,0.105854,0.075943,0.068029,0.066417,0.050817,0.047021,0.060383
BATTERY,0.105854,1.000000,0.101798,0.139691,0.104406,0.061343,0.049696,0.079825
CRIMINAL DAMAGE,0.075943,0.101798,1.000000,0.074156,0.061469,0.051847,0.038608,0.047310
NARCOTICS,0.068029,0.139691,0.074156,1.000000,0.086472,0.047265,0.044761,0.069765
ASSAULT,0.066417,0.104406,0.061469,0.086472,1.000000,0.036869,0.031553,0.050848
BURGLARY,0.050817,0.061343,0.051847,0.047265,0.036869,1.000000,0.028230,0.033261
MOTOR VEHICLE THEFT,0.047021,0.049696,0.038608,0.044761,0.031553,0.028230,1.000000,0.029594
ROBBERY,0.060383,0.079825,0.047310,0.069765,0.050848,0.033261,0.029594,1.000000


In [66]:
X = dense_crimes[:-1,:-1].swapaxes(1,2).reshape(-1,9)
y = dense_crimes[1:,0].reshape(-1)
y[y > 0] = 1
from sklearn.feature_selection import mutual_info_classif

In [67]:
mutual_info_classif(X,y)

array([0.07804144, 0.02078052, 0.02245209, 0.01046461, 0.01601758,
       0.0073597 , 0.00517525, 0.00221414, 0.00494581])

# Why conditional mutual information is larger than mutual information
The conditional variable has more possible states and therfore the join has a higher entropy - the maximum value it can take on can therefore be higher - to make plots easier to read we can normalize the mutual information with the mutual information of the target signal with itself.

$I(X;Y) = H(X) + H(Y) - H(X,Y)$


$I(X;Y|Z) = H(X,Z) + H(Y,Z) - H(X,Y,Z) - H(Z)$

In [1]:
from sparse_discrete_table import SparseDiscreteTable, build_discrete_table
from ipywidgets import widgets
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import plotly.graph_objects as go

In [2]:
def plot(y,x=None, mode='lines+markers'):
    if x is None or len(x) != len(y):
        x = np.arange(len(y))
        
    data = [go.Scatter(x=y,y=y,mode=mode)]
    layout = dict(width=900,height=400)
    return go.FigureWidget(data=data, layout=layout)

In [3]:
def gen_obs(k,mu,w,n):
    obs = []
    for i in range(n):
        lam = mu+k
        try:
            o = poisson.rvs(mu=lam,size=1)
            obs.append(o[0])
            k = k*(1-w) + o*(w)
        except ValueError as e:
            raise ValueError(f"{e} - lam={lam}")
            
    return obs


def gen_mi(obs):
    mis = []
    for t in range(1,30):
        xy = np.stack([obs[t:], obs[:-t]], axis=1)
        dt = build_discrete_table(xy, ['x','y'])
        mi = dt.mutual_information(['x'], ['y'])
    #     print(f"mi: {mi}")
        mis.append(mi)
    return mis

In [4]:
from scipy.stats import poisson

n = 100
mu = .05
k = 0
w = .91

obs = gen_obs(k,mu,w,n)
fig = plot(gen_mi(obs))
fig_obs = plot(obs)


k_slider = widgets.FloatSlider(
    value=k,
    min=0,
    max=10.0,
    step=1,
    description='k',
    continuous_update=False,
)

mu_slider = widgets.FloatSlider(
    value=mu,
    min=.05,
    max=2.0,
    step=0.05,
    description='mu',
    continuous_update=False,
)

w_slider = widgets.FloatSlider(
    value=w,
    min=.001,
    max=.99,
    step=.001,
    description='w',
    continuous_update=False,
)

n_slider = widgets.IntSlider(
    value=n,
    min=1,
    max=10_000,
    step=10,
    description='n',
    continuous_update=False,
)


wl = [k_slider, mu_slider, w_slider, n_slider]
container = widgets.VBox(wl)

def regiser_callback(fn, widget_list):
    for widg in widget_list:
        widg.observe(fn)
        
def callback(change):
    global k, mu, w, n, fig
#     print(f"change: {change}\n\n")
    if isinstance(change, dict) and change['name'] == 'value':
        val = change['new']
        desc = change['owner'].description
        if desc == 'k':
            k = val
        if desc == 'mu':
            mu = val
        if desc == 'w':
            w = val
        if desc == 'n':
            n = val  
            
        y_obs = gen_obs(k,mu,w,n)
        x_obs = np.arange(len(y_obs))
        
        y = gen_mi(obs)
        x = np.arange(len(y))
        with fig.batch_update():
            fig.data[0].x = x
            fig.data[0].y = y
        
        with fig.batch_update():
            fig_obs.data[0].x = x_obs
            fig_obs.data[0].y = y_obs
        
regiser_callback(callback, wl)
widgets.VBox([container, fig, fig_obs])

# Interactive Plotly Plots

In [1]:
from pprint import pprint

In [2]:
mapbox_access_token = open(".mapbox_token").read()
# mapbox_access_token = "open-street-map"
mapbox_access_token

on_state = ('#f050ae', 8)
off_state = ('#ffab00', 8)

mapbox_styles = {
    "sat": "mapbox://styles/bernsblack/ckecz0wr52pfc1at7tvu43fmj", 
    "mono": "mapbox://styles/bernsblack/ckecyyizy065w19psrikmeo5d",
    "dark": "mapbox://styles/bernsblack/ckeikbchd254619s57np6iyum",
}

In [3]:
from utils.data_processing import time_series_to_time_index

In [4]:
# importing ogirinal data
import pandas as pd
import numpy as np
import plotly.graph_objs as go

In [5]:
color_palette = ["#33a8c7","#52e3e1","#a0e426","#fdf148","#ffab00","#f77976","#f050ae","#d883ff","#9336fd"]

In [6]:
df = pd.read_csv("data/original/Crimes_Chicago_2001_to_2019.csv",nrows=100_000)

In [27]:
df = df[[
    'ID',
    'Date',
    'Primary Type',
#     'Arrest',
    'Latitude',
    'Longitude',
#     'Census Tracts',
]].dropna()

In [28]:
# filter out rare crime categories
top_k_crimes_cats = len(color_palette)
crime_categories = list(df['Primary Type'].value_counts().index)[:top_k_crimes_cats]
df = df[df['Primary Type'].isin(crime_categories)]

In [29]:
from pandas.api.types import CategoricalDtype
# Label encode the crime catgegories - makes histograms faster
CrimeType = CategoricalDtype(categories=crime_categories, ordered=True)
df['Primary Type'] = df['Primary Type'].astype(CrimeType)
df['c'] = df['Primary Type'].cat.codes

In [30]:
# convert string date to date time - takes the most time
df['Date'] = pd.DatetimeIndex(pd.to_datetime(df['Date']))

In [31]:
df['t'] = time_series_to_time_index(t_series=df.Date, t_step='1D', floor=True)
# df['t'] = time_series_to_time_index(t_series=df.Date, t_step='1H', floor=True)

In [32]:
from utils.utils import ffloor, fceil

In [33]:
from geopy import distance

# get meter per degree estimat
coord_series = df[['Latitude', 'Longitude']]

lat_min, lon_min = coord_series.min()
lat_max, lon_max = coord_series.max()

lat_mean, lon_mean = coord_series.mean()

dy = distance.distance((lat_min, lon_min), (lat_max, lon_min)).m
dx = distance.distance((lat_min, lon_min), (lat_min, lon_max)).m

lat_per_metre = (lat_max - lat_min)/dy
lon_per_metre = (lon_max - lon_min)/dx

ratio_xy = dx/dy
print(f"ratio_xy: {ratio_xy}")
print(f"lat_per_metre: {lat_per_metre}")
print(f"lon_per_metre: {lon_per_metre}")

ratio_xy: 0.7730030319345187
lat_per_metre: 9.003326876598671e-06
lon_per_metre: 1.2003347847229684e-05


### Very important  - all the meta data is mapped from intervals of 0.001 in the lat and lon space with ratios of 8 and 11 to ensure that the grids cels are square

In [170]:
# [20,16] - [10,8] - [5,4] - [3,2] - [1,1]
x_scale, y_scale = 5,4#3,2
xy_scale = np.array([x_scale, y_scale])  # must be integer so that we can easily sample demographic data
dlon, dlat = xy_scale * np.array([0.001, 0.001])
meta_info = {}
meta_info["x_scale"] = x_scale
meta_info["y_scale"] = y_scale
meta_info["dlon"] = float(dlon)
meta_info["dlat"] = float(dlat)
meta_info["x in metres"] = 85000 * float(dlon)
meta_info["y in metres"] = 110000 * float(dlat)

pprint(meta_info)

# use perfect squares perfect squares
"""
all the meta data is mapped from intervals of 0.001 in 
the lat and lon space with ratios of 8 and 11 to ensure that the grids cels are square
"""
# cell_size_m = 430
# dlat = cell_size_m*lat_per_metre
# dlon = cell_size_m*lon_per_metre
 
# use increments of 0.001

xbins = np.arange(ffloor(lon_min, dlon),fceil(lon_max,dlon), dlon)
nx = len(xbins)

ybins = np.arange(ffloor(lat_min, dlat),fceil(lat_max,dlat), dlat)
ny = len(ybins)

nt = int(np.ceil(df['t'].max()))
tbins = np.arange(int(np.floor(df['t'].min())),nt+1,1)

nc = len(crime_categories)
cbins = np.arange(0,nc+1,1)

{'dlat': 0.004,
 'dlon': 0.005,
 'x in metres': 425.0,
 'x_scale': 5,
 'y in metres': 440.0,
 'y_scale': 4}


In [171]:
counts, edges = np.histogramdd(
    sample=df[['t', 'c', 'Latitude', 'Longitude']].values,
    bins=(tbins, cbins, ybins, xbins),
) # (nt, nc, ny, nx))

tbins, cbins, ybins, xbins = edges

counts_sum  = counts.sum(1)
counts_mean = counts_sum.mean(0)

counts.shape

(136, 9, 95, 79)

In [172]:
# todo get better init values or save in state object
lon_filter = []
lat_filter = []
date_range_filter = ()
selected_crime_types = []
current_date_label = "Empty"
time_index = 0

In [173]:
from utils.plots import hist2d_to_geo

In [174]:
blocks, values = hist2d_to_geo(counts_mean,xbins,ybins,filter_zero=True)

In [175]:
sub = df.sample(1000)

### Setup plots

In [176]:
heatmap = go.Choroplethmapbox(
    geojson=blocks,
    locations=values.id,
    z=values.value,
    colorscale="Viridis",
    zmin=values.value.min(),
    zmax=values.value.max(),
    marker_opacity=0.5,
    marker_line_width=0,
    name='heatmap',
    hoverinfo='text',
    hovertext=values[['y','x','value']],
)

scatter = go.Scattermapbox(
    lon=sub.Longitude,
    lat=sub.Latitude,
    opacity=.4,
    hoverinfo='skip',
    name='scatter',
)

fig = go.FigureWidget(data=[heatmap, scatter])

fig.update_layout(
    clickmode='event+select',
    mapbox_style=mapbox_styles["sat"],
    mapbox_accesstoken=mapbox_access_token,
    mapbox_zoom=10,
    mapbox_center = {"lat": np.mean(ybins), "lon": np.mean(xbins)},
    margin={"r":0,"t":30,"l":0,"b":10},
    height=500,
#     showlegend=True,
#     legend=dict(
#         yanchor="top",
#         y=0.99,
#         xanchor="left",
#         x=0.01,
#     ),
)

def heatmap_on_select(trace,points,selector):
    global lat_filter, lon_filter
    
    inds = points.point_inds
    selection = values.iloc[inds]
    _,_,f_x_min,f_y_min = selection.min()
    _,_,f_x_max,f_y_max = selection.max()

    lon_filter = xbins[[f_x_min,f_x_max]]
    lat_filter = ybins[[f_y_min,f_y_max]]
    
    
heatmap = fig.data[0]    
# heatmap.on_click(callback_fn)
heatmap.on_selection(heatmap_on_select)

fig

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

In [177]:
scattergl = go.Scattergl(
    x=sub.Longitude,
    y=sub.Latitude,
    opacity=.6,
    hoverinfo='skip',
    mode='markers',
    name='scattergl',
#     marker_symbol="x-thin", # "cross",
    marker_color='red',
#     hovertext="",
)

heatmapgl = go.Heatmap(
    z=counts_mean,
    x=xbins,
    y=ybins, 
    hoverinfo='skip',
    colorscale='viridis',
    opacity=1,
    name='heatmapgl',
)


fig_height = 600

fig2 = go.FigureWidget(
    data=[heatmapgl, scattergl],
    layout=dict(
        margin={"r":50,"t":30,"l":0,"b":0},
        height=fig_height,
#         width=fig_height*ratio_xy,
        yaxis=dict(scaleanchor="x", scaleratio=1/ratio_xy),
        clickmode='event+select',
    ),
)

scattergl = fig2.data[1]

In [178]:
import plotly.express as px


# series = sub.groupby(by=['t'])['ID'].count().reset_index(name="Count") # too many missing values - use hist
fline = go.FigureWidget(px.line(data_frame=series, x="t",y='Count'))

In [179]:
# global updater
def update_data():
    global df, sub, lon_filter, lat_filter, date_range_filter, time_index
    
    # spatial filters
    if lon_filter:
        f_lon_min, f_lon_max = lon_filter
        lon_mask = (df.Longitude >= f_lon_min) & (df.Longitude <= f_lon_max)
    
    if lat_filter:
        f_lat_min, f_lat_max = lat_filter
        lat_mask = (df.Latitude >= f_lat_min) & (df.Latitude <= f_lat_max)
    
    
    # time filters
    f_date_min, f_date_max = date_range[[*date_range_filter]]
    date_range_mask = (df.Date >= f_date_min) & (df.Date <= f_date_max) 
    
    # crime type filters
    crime_types_mask = df['Primary Type'].isin(selected_crime_types)
    
    selected_mask = date_range_mask & crime_types_mask
    
    if lat_filter and lon_filter:
        selected_mask = selected_mask & lon_mask & lat_mask
    
    sub = df[selected_mask]
    
#     series = sub.groupby(by=['t'])['ID'].count().reset_index(name="Count") # dont use - too mayn missing values

    # todo implement histo
#     counts, edges = np.histogramdd(
#         sample=sub[['t', 'c', 'Latitude', 'Longitude']].values,
#         bins=(tbins, cbins, ybins, xbins),
#     )

    
    nt = int(np.ceil(sub['t'].max()))
    tbins = np.arange(int(np.floor(sub['t'].min())),nt+1,1)

    
    with fig2.batch_update():
        scattergl.x=sub.Longitude
        scattergl.y=sub.Latitude
        
        fline.data[0].x = series.t
        fline.data[0].y = series["Count"]

In [188]:
nt = int(np.ceil(sub['t'].max()))
tbins = np.arange(int(np.floor(sub['t'].min())),nt+1,1)

np.histogram(sub.t, tbins)

(array([34, 51, 62, 66, 62, 55, 55, 35, 72, 69, 51, 59, 46, 46, 56, 49, 55,
        60, 49, 56, 40, 47, 69, 66, 55, 52, 53, 51, 48, 52, 53, 54, 32, 58,
        62, 46, 65, 59, 56, 47, 49, 51, 65, 59, 73, 65, 49, 50, 73, 57, 71,
        61, 69, 42, 70, 59, 61, 63, 57, 80, 59, 67, 53, 46, 65, 78, 69, 66,
        57, 49, 66, 61, 70, 74, 57, 55, 51, 52, 69, 66, 76, 93, 67, 55, 62,
        54, 85, 81, 60, 60, 53, 67, 57, 56, 77, 56, 63, 66, 65, 81, 62, 59,
        64, 59, 62, 62, 44, 59, 52, 64, 55, 57, 72, 68, 66, 57, 69, 70, 56,
        67, 80, 58, 96]),
 array([  6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
         19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
         45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
         58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
         71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
    

In [180]:
def scattergl_on_select(trace,points,selector):
    global lat_filter, lon_filter
    if trace.name == 'scattergl':                
        lon_filter = selector.xrange
        lat_filter = selector.yrange
        
        update_data()
        
    
scattergl.on_selection(scattergl_on_select)

### Setup widgets
TODO: split widgets into 2 types:
- Filter base data
- Select data (point in time and space)

In [181]:
# widget setup
from ipywidgets import Layout, widgets

# helper functions
def get_widget_index(change):
    if isinstance(change, dict) and change.get('name') == 'index':
        return change.get('new')
    return None

def get_widget_value(change):
    if isinstance(change, dict) and change.get('name') == 'value':
        return change.get('new')
    return None


# date selectors
date_range = pd.date_range(df.Date.min().ceil('1D'), df.Date.max().floor('1D'), freq='1D')

date_range_filter = [date_range.min(),date_range.max()]

date_range_slider = widgets.SelectionRangeSlider(
    options=[d.strftime('%y/%m/%d') for d in date_range],
    index=(0, len(date_range)-1),
    description='Date Range:',
    disabled=False,
    orientation = 'horizontal',
    layout=Layout(width='95%'),
    continuous_update=False,
)

def set_date_range_filter(change):
    global date_range_filter
    
    index = get_widget_index(change)
    if index:
        date_range_filter = index
        update_data()

date_range_slider.observe(set_date_range_filter)


# date display label
current_date_label = widgets.Label(f'Date: {date_range[0]}')

# time index selector
time_index_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(date_range)-1,
    step=1,
    description='Time Index:',
    continuous_update=False,
    layout=Layout(width='80%'),
)

def time_index_change(change):
    global current_date_label, time_index
    time_index = get_widget_value(change)
    if time_index:
        current_date_label.value = f'Date: {date_range[time_index]}'
        
time_index_slider.observe(time_index_change)

play_button = widgets.Play(
    value=0,
    min=time_index_slider.min,
    max=time_index_slider.max,
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)

widgets.jslink((play_button, 'value'), (time_index_slider, 'value'))

time_selectors = widgets.VBox([
    widgets.HBox([date_range_slider]),
    widgets.HBox([time_index_slider])    
])


# crime type selector
selected_crime_types = []

def set_selected_crime_types(button):
    global selected_crime_types
    if button.button_style == "":
        button.button_style = "success"
        selected_crime_types.append(button.description)
    else:
        button.button_style = ""
        selected_crime_types.remove(button.description)
    update_data()
    
def new_buttons(names,all_selected=False):
    global selected_crime_types
    buttons = []
    
    if all_selected:
        selected_crime_types = [*names]
        button_style = "success"
    else:
        selected_crime_types = []
        button_style = ""
        
    for name in names:
        button = widgets.Button(
            description=name,
            tooltip=name,
            disabled=False,
            button_style=button_style, # 'success', 'info', 'warning', 'danger' or ''
        )
        
        button.on_click(set_selected_crime_types)
        
        buttons.append(button)
    return buttons
    
buttons = new_buttons(names=crime_categories, all_selected=True)
crime_selectors = widgets.VBox([
    widgets.HBox(buttons[:3]),
    widgets.HBox(buttons[3:6]),
    widgets.HBox(buttons[6:]),
])


# widgets combinations and setup
row0 = widgets.HBox([play_button, crime_selectors])
row1 = time_selectors

controller = widgets.VBox([row0, row1,current_date_label])

In [182]:
widgets.VBox([fline, controller, fig2])

    'data': [{'hovertemplate': 't=%{x}<br>Count=%{y}<extra></extra>',
          …

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

...

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

IndexError: arrays used as indices must be of integer or boolean type

---
### Animated Crime Over Time

In [745]:
def new_ffloor(delta):
    def res(value): 
        return ffloor(value, delta)
    return res


# df['t'] = time_series_to_time_index(t_series=df.Date, t_step='4H', floor=True)
df['t'] = time_series_to_time_index(t_series=df.Date, t_step='3H', floor=True)


sub = df[['t', 'Longitude', 'Latitude','c', 'Primary Type', 'Date']].copy()
sub['date_str'] = sub.Date.dt.floor('1D').dt.strftime('%c')
sub['count'] = np.ones_like(sub.t)
ff = new_ffloor(0.01)
sub[['Longitude', 'Latitude']] = sub[['Longitude', 'Latitude']].apply(ff)

gb = sub.groupby(by=['Longitude', 'Latitude','t','c'])
points = gb.first()
points['count'] = gb.count()['count']
# points['count'] = points['count'].apply(lambda x: np.log2(1+x))
points['count'] = points['count']/points['count'].min()
points = points.reset_index()
points = points.sort_values(['t','c'])

points = points[points.t < 4*5]
points

,Longitude,Latitude,t,c,Primary Type,Date,date_str,count
464,-87.78,41.79,0,0,THEFT,2019-07-02 01:00:00,Tue Jul 2 00:00:00 2019,1.0
3237,-87.73,41.94,0,0,THEFT,2019-07-02 02:00:00,Tue Jul 2 00:00:00 2019,1.0
3656,-87.72,41.87,0,0,THEFT,2019-07-02 01:10:00,Tue Jul 2 00:00:00 2019,1.0
5518,-87.70,41.95,0,0,THEFT,2019-07-02 02:00:00,Tue Jul 2 00:00:00 2019,1.0
5528,-87.70,41.96,0,0,THEFT,2019-07-02 01:18:00,Tue Jul 2 00:00:00 2019,1.0
...,...,...,...,...,...,...,...,...
6691,-87.68,41.79,19,7,BURGLARY,2019-07-04 10:00:00,Thu Jul 4 00:00:00 2019,1.0
7844,-87.67,41.82,19,7,BURGLARY,2019-07-04 10:00:00,Thu Jul 4 00:00:00 2019,1.0
1861,-87.75,41.91,19,8,MOTOR VEHICLE THEFT,2019-07-04 10:00:00,Thu Jul 4 00:00:00 2019,1.0
6355,-87.69,41.96,19,8,MOTOR VEHICLE THEFT,2019-07-04 12:00:00,Thu Jul 4 00:00:00 2019,1.0


In [942]:
import plotly.express as px

points['size'] = 2*points['count']

fig = px.scatter(
    data_frame=points, 
    x="Longitude", 
    y="Latitude", 
    title="Animated Crime Over Time",
#     facet_col="Primary Type",
#     facet_col_wrap=3,
    width=800, 
    height=800,
    opacity=.5,
    size="size",
    size_max=points['size'].max(),
    color='Primary Type',
#     marginal_x='violin',
    animation_frame="t",
#     animation_group="t",
#     color="count",
)

fig.update_layout(
    margin=dict(l=20, r=50, t=50, b=20),
    paper_bgcolor="LightSteelBlue",
#     transition ={'duration': 1},
#     frame={'duration': 100},
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 0

fig

---